Design, implement, and demonstrate a complete multi-agent AI system that:
1. Reads user feedback from CSV files containing app store reviews and support
emails
2. Classifies content into categories (Bug / Feature Request / Praise / Complaint /
Spam)
3. Extracts actionable insights and technical details
4. Creates structured tickets and logs them to CSV files with appropriate priority
levels and metadata
5. Ensures quality and consistency through automated review

Note: All outputs are derived solely from the CSV input (feedback text and metadata coming from csv only). The LLM does not fetch external data; it only classifies into different categories, interprets, and structures the given text using language understanding, ensuring no external knowledge sources influence the results.

In [233]:
#import streamlit as st
import pandas as pd
import asyncio
from langgraph.graph import StateGraph
from langchain_groq import ChatGroq
from typing import TypedDict, Optional, Dict, List
import re
from dotenv import load_dotenv
import os
import requests
import logging

# === CONFIG ===
load_dotenv()
#GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_KEY = "7AlZuAPJX8f4D7J9cDWGdyb3FY33pbbdekjY9h1dDBYigyd6E9"
#ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

# # Setup logging : basic logging on terminal
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)


import logging
from logging.handlers import RotatingFileHandler

logger = logging.getLogger("feedback_pipeline")
logger.setLevel(logging.INFO)

if not logger.handlers:   # IMPORTANT
    file_handler = RotatingFileHandler(
        "feedback_pipeline.log",
        maxBytes=5_000_000,
        backupCount=3
    )

    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(name)s - %(message)s"
    )
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)



In [234]:

# === LLM ===

llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

In [235]:

# === CONSTANTS ===
INTENT_DETECTION_NODE = "Intent Detection"

# === STATE ===
################################Changed##############################

class FeedbackState(TypedDict):
    # === INPUT ===
    source_id: str                    # review_id or email_id
    source_type: str                  # "app_review" | "support_email"
    raw_text: str                     # review_text or email body/subject
    metadata: Dict[str, str]          # platform, rating, timestamp, app_version, sender, etc.

    # === FEEDBACK CLASSIFICATION ===
    category: Optional[str]           # Bug | Feature Request | Praise | Complaint | Spam
    confidence: Optional[float]       # Classification confidence score
    priority: Optional[str]           # Critical | High | Medium | Low

    # === BUG/Feature ANALYSIS OUTPUT ===
    technical_details: Optional[str]  # For bugs: device, OS, repro steps, severity
    feature_details: Optional[str]    # For features: description, impact, demand
    analysis_notes: Optional[str]     # Agent reasoning / observations

    # === TICKET CREATION ===
    ticket_title: Optional[str]       # Suggested ticket title
    ticket_description: Optional[str] # Structured ticket body
    ticket_metadata: Optional[Dict[str, str]]  # tags, component, version, etc.

    # === QUALITY CONTROL ===
    qc_status: Optional[str]          # Approved | Needs Review | Rejected
    qc_feedback: Optional[str]        # Critic agent comments

    # === CONTROL & TRACEABILITY ===
    manual_review_flag: Optional[bool]  # Human-in-the-loop trigger
    processing_log: Optional[List[str]] # Step-by-step processing history



# === USER PROFILE COLLECTION ===
# PATTERN: ReAct (LLM extraction = Reason → Update profile = Action)
# MODULE: Perception + Learning (extracts missing profile info)
#async def csv_reader_agent(state: FeedbackState) -> FeedbackState:
#    return state

def csv_reader_agent(
    file_path: str,
    source_type: str
) -> List[FeedbackState]:
    """
    Reads feedback CSV and converts each row into FeedbackState.
    source_type: 'review' or 'email' 
    """

    df = pd.read_csv(file_path)
    print(f'\nThe column names in this csv {file_path} are-->', df.columns)
    feedback_states: List[FeedbackState] = []

    for _, row in df.iterrows():
        if source_type == "review":
            raw_text = row.get("review_text", "")
            source_id = str(row.get("review_id"))
        else:  # support email
            raw_text = f"{row.get('subject', '')}. {row.get('body', '')}"
            source_id = str(row.get("email_id"))

        state: FeedbackState = {
            "input_filename": file_path,
            "source_id": source_id,
            "source_type": source_type,
            "raw_text": raw_text,

            "category": None,
            "confidence": None,
            "priority": None,

            "technical_details": None,
            "feature_details": None,

            "ticket_title": None,
            "ticket_description": None,

            "manual_review_flag": False,
            "qc_feedback": [],

            "processing_log": [
                f"CSV Reader: Loaded {source_type} record {source_id}"
            ]
        }

        feedback_states.append(state)

    logger.info(
        f"Loaded {len(feedback_states)} records from {file_path} as {source_type}"
    )
    print('\n feedback state is/are -->',feedback_states)
    return feedback_states



#async def feedback_classifier_agent(state: FeedbackState) -> FeedbackState:
#    return state

# === FEEDBACK CLASSIFIER AGENT ===
async def feedback_classifier_agent(state: FeedbackState) -> FeedbackState:
    raw_text = state.get("raw_text", "")
    metadata = state.get("metadata", {})

    print('Raw text is -->',raw_text)
    print('Metadat is--->', metadata)


    processing_log: List[str] = state.get("processing_log", [])

    # --- LLM Prompt ---
    prompt = (
        f"Classify the following user feedback into one of: "
        f"'Bug', 'Feature Request', 'Praise', 'Complaint', 'Spam'. "
        f"Provide a confidence score (0-1) and suggest an initial priority "
        f"(Critical, High, Medium, Low).\n\n"
        f"Feedback: {raw_text}\n"
        f"Metadata: {metadata}\n\n"
        f"Format: category: <category>\n"
        f"confidence: <0-1>\n"
        f"priority: <priority>\n"
    )

    # --- Call LLM ---
    response = await llm.ainvoke(prompt)
    message = response.content.strip()
    processing_log.append(f"Classifier prompt: {prompt}")
    processing_log.append(f"Classifier response: {message}")

    # --- Parse LLM response ---
    category_match = re.search(r"category:\s*(\w+)", message, re.IGNORECASE)
    confidence_match = re.search(r"confidence:\s*([0-1]\.?\d*)", message)
    priority_match = re.search(r"priority:\s*(\w+)", message, re.IGNORECASE)

    category = category_match.group(1) if category_match else "Unknown"
    confidence = float(confidence_match.group(1)) if confidence_match else None
    priority = priority_match.group(1) if priority_match else "Medium"

    # --- Update state ---
    state.update({
        "category": category,
        "confidence": confidence,
        "priority": priority,
        "processing_log": processing_log
    })

    return state

#async def bug_analysis_agent(state: FeedbackState) -> FeedbackState:
#    return state

async def bug_analysis_agent(state: FeedbackState) -> FeedbackState:
    if state.get("category") != "Bug":
        # Skip if not a bug
        return state

    raw_text = state.get("raw_text", "")
    metadata = state.get("metadata", {})

    print('\n Raw Data',raw_text)
    print('\n Metadata',metadata)

    processing_log: List[str] = state.get("processing_log", [])

    # --- LLM Prompt ---
    prompt = (
        f"Analyze this bug report and extract technical details:\n"
        f"- Device/OS information\n"
        f"- Steps to reproduce\n"
        f"- Severity (Critical/High/Medium/Low)\n"
        f"Provide the output in structured format.\n\n"
        f"Feedback: {raw_text}\n"
        f"Metadata: {metadata}\n"
        f"Format:\n"
        f"device_info: <device info>\n"
        f"os_version: <OS info>\n"
        f"steps_to_reproduce: <steps>\n"
        f"severity: <severity>\n"
    )

    response = await llm.ainvoke(prompt)
    message = response.content.strip()

    processing_log.append(f"Bug Analysis prompt: {prompt}")
    processing_log.append(f"Bug Analysis response: {message}")

    # --- Parse LLM response ---
    device_info_match = re.search(r"device_info:\s*(.*)", message)
    os_version_match = re.search(r"os_version:\s*(.*)", message)
    steps_match = re.search(r"steps_to_reproduce:\s*(.*)", message)
    severity_match = re.search(r"severity:\s*(.*)", message, re.IGNORECASE)

    technical_details = {
        "device_info": device_info_match.group(1) if device_info_match else "",
        "os_version": os_version_match.group(1) if os_version_match else "",
        "steps_to_reproduce": steps_match.group(1) if steps_match else "",
    }

    priority = severity_match.group(1) if severity_match else state.get("priority", "Medium")

    # --- Update state ---
    state.update({
        "technical_details": technical_details,
        "priority": priority,
        "processing_log": processing_log
    })

    return state


#async def feature_extraction_agent(state: FeedbackState) -> FeedbackState:
#    return state
async def feature_extraction_agent(state: FeedbackState) -> FeedbackState:
    if state.get("category") != "Feature Request":
        # Skip if not a feature request
        return state

    raw_text = state.get("raw_text", "")
    metadata = state.get("metadata", {})
    processing_log: List[str] = state.get("processing_log", [])

    # --- LLM Prompt ---
    prompt = (
        f"Analyze this feature request and extract actionable details:\n"
        f"- Feature description\n"
        f"- User intent / goal\n"
        f"- Estimated user impact / demand\n"
        f"Provide the output in structured format.\n\n"
        f"Feedback: {raw_text}\n"
        f"Metadata: {metadata}\n"
        f"Format:\n"
        f"feature_description: <description>\n"
        f"user_intent: <intent>\n"
        f"user_impact: <high/medium/low>\n"
    )

    # --- Call LLM ---
    response = await llm.ainvoke(prompt)
    message = response.content.strip()

    processing_log.append(f"Feature Extraction prompt: {prompt}")
    processing_log.append(f"Feature Extraction response: {message}")

    # --- Parse LLM response ---
    description_match = re.search(r"feature_description:\s*(.*)", message)
    intent_match = re.search(r"user_intent:\s*(.*)", message)
    impact_match = re.search(r"user_impact:\s*(.*)", message, re.IGNORECASE)

    feature_details = {
        "feature_description": description_match.group(1) if description_match else "",
        "user_intent": intent_match.group(1) if intent_match else "",
        "user_impact": impact_match.group(1) if impact_match else "Medium"
    }

    # --- Update state ---
    state.update({
        "feature_details": feature_details,
        "processing_log": processing_log
    })

    return state



#async def ticket_creation_agent(state: FeedbackState) -> FeedbackState:
#    return state

#ticket agent which combines outputs from Bug Analysis or Feature Extraction into structured tickets.
async def ticket_creation_agent(state: FeedbackState) -> FeedbackState:
    category = state.get("category", "")
    priority = state.get("priority", "Medium")
    technical_details = state.get("technical_details", {})
    feature_details = state.get("feature_details", {})
    processing_log: List[str] = state.get("processing_log", [])

    # --- Construct ticket ---
    if category == "Bug":
        ticket_title = f"[BUG] {technical_details.get('device_info', '')} - Issue"
        ticket_description = (
            f"Category: {category}\n"
            f"Priority: {priority}\n"
            f"Device Info: {technical_details.get('device_info', '')}\n"
            f"OS Version: {technical_details.get('os_version', '')}\n"
            f"Steps to Reproduce: {technical_details.get('steps_to_reproduce', '')}\n"
        )
    elif category == "Feature Request":
        ticket_title = f"[FEATURE REQUEST] {feature_details.get('feature_description', '')[:50]}"
        ticket_description = (
            f"Category: {category}\n"
            f"Priority: {priority}\n"
            f"Feature Description: {feature_details.get('feature_description', '')}\n"
            f"User Intent: {feature_details.get('user_intent', '')}\n"
            f"Estimated User Impact: {feature_details.get('user_impact', 'Medium')}\n"
        )
    else:
        ticket_title = f"[{category.upper()}] Feedback"
        ticket_description = f"Category: {category}\nPriority: {priority}\n"

    # Include metadata
    metadata = state.get("metadata", {})
    for key, value in metadata.items():
        ticket_description += f"{key}: {value}\n"

    processing_log.append(f"Ticket Created: {ticket_title}")

    import csv
    from typing import List
    # --- CSV Logging ---
    ticket_row = {
        "ticket_title": ticket_title,
        "ticket_description": ticket_description,
        "category": category,
        "priority": priority,
        "technical_details": str(technical_details),
        "feature_details": str(feature_details),
        "metadata": str(metadata)
    }

    #csv_file = f"generated_tickets_" + state["input_filename"]
    csv_file = f"generated_tickets.csv"
    fieldnames = list(ticket_row.keys())

    # Append to CSV (write header if empty)
    with open(csv_file, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if f.tell() == 0:
            writer.writeheader()
        writer.writerow(ticket_row)


    # --- Update state ---
    state.update({
        "ticket_title": ticket_title,
        "ticket_description": ticket_description,
        "ticket_metadata": metadata,
        "processing_log": processing_log
    })

    return state




#async def quality_critic_agent(state: FeedbackState) -> FeedbackState:
#    return state

#Quality Critic Agent, which validates ticket completeness, accuracy, and flags for human review if needed.
async def quality_critic_agent(state: FeedbackState) -> FeedbackState:
    ticket_title = state.get("ticket_title", "")
    ticket_description = state.get("ticket_description", "")
    category = state.get("category", "")
    confidence = state.get("confidence", 1.0)
    processing_log: List[str] = state.get("processing_log", [])

    manual_review_flag = False
    qc_feedback = []

    # --- Basic checks ---
    if not ticket_title:
        manual_review_flag = True
        qc_feedback.append("Ticket title missing.")
    if not ticket_description:
        manual_review_flag = True
        qc_feedback.append("Ticket description missing.")
    if category in ["Bug", "Feature Request"] and confidence is not None and confidence < 0.7:
        manual_review_flag = True
        qc_feedback.append(f"Low confidence ({confidence}) for category {category}.")

    if manual_review_flag:
        processing_log.append(f"Quality Critic flagged manual review: {qc_feedback}")
    else:
        processing_log.append("Quality Critic passed: ticket looks complete.")

    # --- Update state ---
    state.update({
        "manual_review_flag": manual_review_flag,
        "qc_feedback": qc_feedback,
        "processing_log": processing_log
    })

    return state

async def end_node(state: FeedbackState) -> FeedbackState:
    return state


# === BUILD GRAPH ===
# PATTERN: Planning Pattern — router decides next node
# MODULE: Cognition
def get_next_node(state: FeedbackState) -> str:
    if state.get("manual_review_flag"):
        return "Human Review"

    category = state.get("category")

    if category == "Bug":
        return "Bug Analysis"
    elif category == "Feature Request":
        return "Feature Extraction"
    elif category in ["Praise", "Complaint"]:
        return "Ticket Creation"
    elif category == "Spam":
        return "End"
    else:
        return "End"


#

In [236]:

# run csv reader agent

In [237]:
states =  csv_reader_agent("app_store_reviews.csv", "review")

state = states[0]  # pick one record

print("STEP 1: RAW INPUT")
print(state["raw_text"])

INFO:feedback_pipeline:Loaded 14 records from app_store_reviews.csv as review



The column names in this csv app_store_reviews.csv are--> Index(['review_id', 'platform', 'rating', 'review_text', 'user_name', 'date',
       'app_version'],
      dtype='object')

 feedback state is/are --> [{'input_filename': 'app_store_reviews.csv', 'source_id': 'R001', 'source_type': 'review', 'raw_text': 'App crashes when I try to open my task list after the latest update.', 'category': None, 'confidence': None, 'priority': None, 'technical_details': None, 'feature_details': None, 'ticket_title': None, 'ticket_description': None, 'manual_review_flag': False, 'qc_feedback': [], 'processing_log': ['CSV Reader: Loaded review record R001']}, {'input_filename': 'app_store_reviews.csv', 'source_id': 'R002', 'source_type': 'review', 'raw_text': "Can't login since update. Keeps showing an error on iOS 17.", 'category': None, 'confidence': None, 'priority': None, 'technical_details': None, 'feature_details': None, 'ticket_title': None, 'ticket_description': None, 'manual_review_flag': Fa

Feedback classifier Agent

#Note: becase function is async that is parallel processing so await keyword required


In [238]:

state = await feedback_classifier_agent(state)
print("\nSTEP 2: CLASSIFICATION")

print(state)

print('State is ->',state["category"],'& confidence is', state["confidence"])


Raw text is --> App crashes when I try to open my task list after the latest update.
Metadat is---> {}


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



STEP 2: CLASSIFICATION
{'input_filename': 'app_store_reviews.csv', 'source_id': 'R001', 'source_type': 'review', 'raw_text': 'App crashes when I try to open my task list after the latest update.', 'category': 'Bug', 'confidence': 1.0, 'priority': 'Critical', 'technical_details': None, 'feature_details': None, 'ticket_title': None, 'ticket_description': None, 'manual_review_flag': False, 'qc_feedback': [], 'processing_log': ['CSV Reader: Loaded review record R001', "Classifier prompt: Classify the following user feedback into one of: 'Bug', 'Feature Request', 'Praise', 'Complaint', 'Spam'. Provide a confidence score (0-1) and suggest an initial priority (Critical, High, Medium, Low).\n\nFeedback: App crashes when I try to open my task list after the latest update.\nMetadata: {}\n\nFormat: category: <category>\nconfidence: <0-1>\npriority: <priority>\n", 'Classifier response: category: Bug\nconfidence: 1\npriority: Critical\n\nThe user is reporting an issue where the app crashes, which 

Conditional Analysis

In [239]:
# STEP 3: Conditional analysis
if state["category"] == "Bug":
    state = await bug_analysis_agent(state)
    print("\nBug details:", state["technical_details"])

elif state["category"] == "Feature Request":
    state = await feature_extraction_agent(state)
    print("\nFeature details:", state["feature_details"])


 Raw Data App crashes when I try to open my task list after the latest update.

 Metadata {}


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Bug details: {'device_info': '', 'os_version': '', 'steps_to_reproduce': ''}


Ticket Creation

In [240]:
state = await ticket_creation_agent(state)
print("\nSTEP 4: TICKET")
print(state["ticket_title"])

print('The state so far now is -->')
state


STEP 4: TICKET
[BUG]  - Issue
The state so far now is -->


{'input_filename': 'app_store_reviews.csv',
 'source_id': 'R001',
 'source_type': 'review',
 'raw_text': 'App crashes when I try to open my task list after the latest update.',
 'category': 'Bug',
 'confidence': 1.0,
 'priority': 'High (since the app crashes, causing inconvenience to the user and potentially data loss)',
 'technical_details': {'device_info': '',
  'os_version': '',
  'steps_to_reproduce': ''},
 'feature_details': None,
 'ticket_title': '[BUG]  - Issue',
 'ticket_description': 'Category: Bug\nPriority: High (since the app crashes, causing inconvenience to the user and potentially data loss)\nDevice Info: \nOS Version: \nSteps to Reproduce: \n',
 'manual_review_flag': False,
 'qc_feedback': [],
 'processing_log': ['CSV Reader: Loaded review record R001',
  "Classifier prompt: Classify the following user feedback into one of: 'Bug', 'Feature Request', 'Praise', 'Complaint', 'Spam'. Provide a confidence score (0-1) and suggest an initial priority (Critical, High, Medium, L

In [241]:
state = await quality_critic_agent(state)
print("\nSTEP 5: QC")
print(state["qc_feedback"])


STEP 5: QC
[]


In [242]:
state

{'input_filename': 'app_store_reviews.csv',
 'source_id': 'R001',
 'source_type': 'review',
 'raw_text': 'App crashes when I try to open my task list after the latest update.',
 'category': 'Bug',
 'confidence': 1.0,
 'priority': 'High (since the app crashes, causing inconvenience to the user and potentially data loss)',
 'technical_details': {'device_info': '',
  'os_version': '',
  'steps_to_reproduce': ''},
 'feature_details': None,
 'ticket_title': '[BUG]  - Issue',
 'ticket_description': 'Category: Bug\nPriority: High (since the app crashes, causing inconvenience to the user and potentially data loss)\nDevice Info: \nOS Version: \nSteps to Reproduce: \n',
 'manual_review_flag': False,
 'qc_feedback': [],
 'processing_log': ['CSV Reader: Loaded review record R001',
  "Classifier prompt: Classify the following user feedback into one of: 'Bug', 'Feature Request', 'Praise', 'Complaint', 'Spam'. Provide a confidence score (0-1) and suggest an initial priority (Critical, High, Medium, L

In [243]:
# # OR creating whole pipleine 

# async def debug_pipeline():
#     states = csv_reader_agent("app_store_reviews.csv", "review")
#     state = states[0]

#     print("STEP 1: RAW INPUT")
#     print(state["raw_text"])

#     state = await feedback_classifier_agent(state)
#     print("\nSTEP 2: CLASSIFICATION")
#     print(state["category"], state["confidence"])

#     if state["category"] == "Bug":
#         state = await bug_analysis_agent(state)
#         print("\nSTEP 3: BUG ANALYSIS")
#         print(state["technical_details"])

#     if state["category"] == "Feature Request":
#         state = await feature_extraction_agent(state)
#         print("\nSTEP 3: FEATURE EXTRACTION")
#         print(state["feature_details"])

#     state = await ticket_creation_agent(state)
#     print("\nSTEP 4: TICKET")
#     print(state["ticket_title"])

#     state = await quality_critic_agent(state)
#     print("\nSTEP 5: QC")
#     print(state["qc_feedback"])

# # Run it
# await debug_pipeline()

Graph Builder / Orchestration step in LangGraph.

In [244]:

builder = StateGraph(FeedbackState)

# PATTERN: Planning — adds modular nodes
# Add nodes
builder.add_node("CSV Reader", csv_reader_agent)
builder.add_node("Feedback Classifier", feedback_classifier_agent)
builder.add_node("Bug Analysis", bug_analysis_agent)
builder.add_node("Feature Extraction", feature_extraction_agent)
builder.add_node("Ticket Creation", ticket_creation_agent)
builder.add_node("Quality Critic", quality_critic_agent)
builder.add_node("End", end_node)

# PATTERN: Planning + ReAct orchestration (routing based on reasoning)
# Entry point #we can 
#Note: This decides the entry point for builder . Cell 93 will exceute everythign from here 
#builder.set_entry_point("CSV Reader")
builder.set_entry_point("Feedback Classifier")

# Edges
builder.add_edge("CSV Reader", "Feedback Classifier")

#This defines dynamic routing in the graph. After the Feedback Classifier runs, get_next_node examines 
# the state and decides which agent should run next (Bug Analysis, Feature Extraction, Ticket Creation, or End),
# based on the classified feedback category.
#In this conditional edge, if get_next_node returns "Bug Analysis", the graph moves to the Bug Analysis node; 
# if it returns "Feature Extraction", it goes there. Keys are decision outputs, values are destination nodes.

builder.add_conditional_edges(
    "Feedback Classifier",
    get_next_node,
    {
        "Bug Analysis": "Bug Analysis",
        "Feature Extraction": "Feature Extraction",
        "Ticket Creation": "Ticket Creation",
        "End": "End"
    }
)

# Post-analysis flow
builder.add_edge("Bug Analysis", "Ticket Creation")
builder.add_edge("Feature Extraction", "Ticket Creation")
builder.add_edge("Ticket Creation", "Quality Critic")
builder.add_edge("Quality Critic", "End")

feedback_graph = builder.compile()



In [245]:
#✔️ astream() only takes state (and optional config), not node inputs.
#async for event in feedback_graph.astream(state):
#    print(event)

Runner

In [246]:
async def run_pipeline(csv_path: str, source_type: str):
    records = csv_reader_agent(csv_path, source_type)

    results = []
    for state in records:
        final_state = await feedback_graph.ainvoke(state)
        results.append(final_state)

    return results



output = await run_pipeline(
    csv_path="support_emails.csv",
    source_type="email"

    #csv_path="app_store_reviews.csv",
    #source_type = "review"
)

for item in output:
    print(item["source_id"], item["category"])
#if __name__ == "__main__":
    # output = asyncio.run(
    #     run_pipeline(
    #         csv_path="support_emails.csv",
    #         source_type="email"   # or "review"
    #     )
    # )

    # for item in output:
    #     print("=" * 50)
    #     print("ID:", item["source_id"])
    #     print("Category:", item["category"])
    #     print("Priority:", item["priority"])
    #     print("Ticket Title:", item["ticket_title"])
    #     print("Manual Review:", item["manual_review_flag"])


INFO:feedback_pipeline:Loaded 9 records from support_emails.csv as email



The column names in this csv support_emails.csv are--> Index(['email_id', 'subject', 'body', 'sender_email', 'timestamp', 'priority'], dtype='object')

 feedback state is/are --> [{'input_filename': 'support_emails.csv', 'source_id': 'E001', 'source_type': 'email', 'raw_text': 'App Crash Report. Dear Team, the app crashes on my Samsung Galaxy S21 running Android 14. Steps: Open app → tap Add Task → crash.', 'category': None, 'confidence': None, 'priority': None, 'technical_details': None, 'feature_details': None, 'ticket_title': None, 'ticket_description': None, 'manual_review_flag': False, 'qc_feedback': [], 'processing_log': ['CSV Reader: Loaded email record E001']}, {'input_filename': 'support_emails.csv', 'source_id': 'E002', 'source_type': 'email', 'raw_text': 'Login Issue. Hi, I cannot login since the last update on my iPhone 14 (iOS 17). Please help.', 'category': None, 'confidence': None, 'priority': None, 'technical_details': None, 'feature_details': None, 'ticket_title': Non

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Raw text is --> Login Issue. Hi, I cannot login since the last update on my iPhone 14 (iOS 17). Please help.
Metadat is---> {}


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...


 Raw Data App Crash Report. App crashes randomly lol. Using Pixel 7, Android 14. Happens when editing tasks.

 Metadata {}


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Raw text is --> Feature Request. Please add recurring tasks feature. Very important for productivity.
Metadat is---> {}
Raw text is --> Spam Inquiry. Check out our marketing services at www.promosite.com
Metadat is---> {}


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


E001 Unknown
E002 Bug
E003 Feature
E004 Feature
E005 Complaint
E006 Praise
E007 Bug
E008 Feature
E009 Spam


Accuracy Metric

In [248]:
import pandas as pd

df_reviews = pd.read_csv("generated_tickets_app_store_reviews.csv")
df_emails = pd.read_csv("generated_tickets._source_email.csv")

df_all = pd.concat([df_reviews, df_emails], ignore_index=True)
df_all.to_csv("generated_tickets_all.csv", index=False)


In [249]:
from sklearn.metrics import classification_report
import pandas as pd

pred = pd.read_csv("generated_tickets_all.csv")["category"]
true = pd.read_csv("expected_classifications.csv")["category"]

print(classification_report(true, pred))

ValueError: Found input variables with inconsistent numbers of samples: [23, 9]